In [1]:
import requests
import pandas as pd
import time
import datetime

#https://stackoverflow.com/questions/49188960/how-to-show-all-columns-names-on-a-large-pandas-dataframe
pd.set_option('display.max_columns', None)

In [2]:
#additional sources: (for readme or w/e)
#https://www.epochconverter.com/

In [3]:
#duilding starting dataframe for first subreddit:
subreddit = 'dndnext'
start_date = '1635356099'
kind = 'submission'

stem = 'https://api.pushshift.io'
slug = f'/reddit/search/{kind}/?subreddit={subreddit}&before={start_date}&size=100'

In [4]:
res = requests.get(stem + slug)

In [5]:
res

<Response [200]>

In [6]:
data = res.json()

In [7]:
#commented out for length
#data['data'][0]

In [8]:
#subreddit author title selftext created utc
posts = []

for post in data['data']:
    post_dict = {
        'subreddit': post['subreddit'],
        'author': post['author'],
        'title': post['title'],
        'selftext': post['selftext'],
        'permalink': post['permalink'],
        'created_utc': post['created_utc'],
    }
    posts.append(post_dict)

In [9]:
pd.DataFrame(posts).head(3)

,subreddit,author,title,selftext,permalink,created_utc
0,dndnext,mellowOats,RAW the Drakewarden can totally use their drak...,So I was reading [this post](https://www.reddi...,/r/dndnext/comments/qh2037/raw_the_drakewarden...,1635356098
1,dndnext,Backflip248,"Does ""True Dragon"" have meaning anymore?",Question is in the title. With Fizban out and ...,/r/dndnext/comments/qh1tyr/does_true_dragon_ha...,1635355614
2,dndnext,coldhandsbigdick,How do people have fun RPing something other t...,I struggle with this *so much*. It's not that ...,/r/dndnext/comments/qh1tvh/how_do_people_have_...,1635355608


In [10]:
pd.DataFrame(posts)['created_utc'].min()

1635279396

In [11]:
pd.DataFrame(posts).to_csv('./csv_files/dndnext.csv', index = False)

- Manually building dataframe for second subreddit:

In [12]:
subreddit_1 = 'dmacademy'

stem = 'https://api.pushshift.io'
slug_1 = f'/reddit/search/{kind}/?subreddit={subreddit_1}&before={start_date}&size=100'

In [13]:
res_1 = requests.get(stem + slug_1)

In [14]:
res_1

<Response [200]>

In [15]:
data_1 = res_1.json()

In [16]:
posts_1 = []

for post in data_1['data']:
    post_dict = {
        'subreddit': post['subreddit'],
        'author': post['author'],
        'title': post['title'],
        'selftext': post['selftext'],
        'permalink': post['permalink'],
        'created_utc': post['created_utc'],
    }
    posts_1.append(post_dict)

In [17]:
pd.DataFrame(posts_1).head(3)

,subreddit,author,title,selftext,permalink,created_utc
0,DMAcademy,TheJackofHats,Non-Combat Travel encounters,Ive been running several sessions of my player...,/r/DMAcademy/comments/qh1w02/noncombat_travel_...,1635355778
1,DMAcademy,El-Ahrairah7,Running an epic encounter where a town is dest...,"Hey all,\nI am about to attempt to run an enco...",/r/DMAcademy/comments/qh1ttr/running_an_epic_e...,1635355603
2,DMAcademy,Hellspore,Worried about my new adventure idea,I was wondering if we have a subreddit where o...,/r/DMAcademy/comments/qh1s9v/worried_about_my_...,1635355485


In [18]:
pd.DataFrame(posts_1)['created_utc'].min()

1635285129

In [19]:
pd.DataFrame(posts_1).to_csv('./csv_files/dmacademy.csv', index = False)

In [20]:
def pushshift(subreddit, kind = 'submission', start_date = '1635356099',
              iters = 10, og_csv = './csv_files/dndnext.csv'):
    
    for scrape in range(1, iters + 1):        
        date = pd.read_csv(og_csv)['created_utc'].min()    
        stem = 'https://api.pushshift.io'
        slug = f'/reddit/search/{kind}/?subreddit={subreddit}&before={date}&size=100&selftext=True'
        
        res = requests.get(stem + slug)
        
        new_file = False
        
        if res.status_code != 200:
            print(f'Error! url: {stem} + {slug} returned {res.status_code}')
            
        else:
            data = res.json()
            posts = []

            for post in data['data']:
                post_dict = {
                    'subreddit': post['subreddit'],
                    'author': post['author'],
                    'title': post['title'],
                    'selftext': post['selftext'],
                    'permalink': post['permalink'],
                    'created_utc': post['created_utc'],
                            }
                posts.append(post_dict)
                
            og_df = pd.read_csv(og_csv)
            temp_df = pd.DataFrame(posts)
            
            new_df = pd.concat([og_df, temp_df])
            
            new_df.to_csv(og_csv, index = False)
        
            print('Scrape success!')
            print(f'Original data was {og_df.shape[0]} rows')
            print(f'New data is {new_df.shape[0]} rows!')
            
            time.sleep(3)

In [21]:
pushshift(subreddit = 'dndnext', kind = 'submission', start_date = '1635356099',
              iters = 49, og_csv = './csv_files/dndnext.csv')

#only 3513 entries so limiting iters to '49' for demonstrability. 

Scrape success!
Original data was 100 rows
New data is 200 rows!
Scrape success!
Original data was 200 rows
New data is 300 rows!
Scrape success!
Original data was 300 rows
New data is 400 rows!
Scrape success!
Original data was 400 rows
New data is 500 rows!
Scrape success!
Original data was 500 rows
New data is 600 rows!
Scrape success!
Original data was 600 rows
New data is 700 rows!
Scrape success!
Original data was 700 rows
New data is 800 rows!
Scrape success!
Original data was 800 rows
New data is 900 rows!
Scrape success!
Original data was 900 rows
New data is 1000 rows!
Scrape success!
Original data was 1000 rows
New data is 1100 rows!
Scrape success!
Original data was 1100 rows
New data is 1200 rows!
Scrape success!
Original data was 1200 rows
New data is 1300 rows!
Scrape success!
Original data was 1300 rows
New data is 1400 rows!
Scrape success!
Original data was 1400 rows
New data is 1500 rows!
Scrape success!
Original data was 1500 rows
New data is 1600 rows!
Scrape succe

In [22]:
pushshift(subreddit = 'dmacademy', kind = 'submission', start_date = '1635356099',
          iters = 49, og_csv = './csv_files/dmacademy.csv')

#only 3090 entries so limiting iters to '49' for demonstrability. 

Scrape success!
Original data was 100 rows
New data is 200 rows!
Scrape success!
Original data was 200 rows
New data is 300 rows!
Scrape success!
Original data was 300 rows
New data is 400 rows!
Scrape success!
Original data was 400 rows
New data is 500 rows!
Scrape success!
Original data was 500 rows
New data is 600 rows!
Scrape success!
Original data was 600 rows
New data is 700 rows!
Scrape success!
Original data was 700 rows
New data is 800 rows!
Scrape success!
Original data was 800 rows
New data is 900 rows!
Scrape success!
Original data was 900 rows
New data is 1000 rows!
Scrape success!
Original data was 1000 rows
New data is 1100 rows!
Scrape success!
Original data was 1100 rows
New data is 1200 rows!
Scrape success!
Original data was 1200 rows
New data is 1300 rows!
Scrape success!
Original data was 1300 rows
New data is 1400 rows!
Scrape success!
Original data was 1400 rows
New data is 1500 rows!
Scrape success!
Original data was 1500 rows
New data is 1600 rows!
Scrape succe

In [23]:
dndnext_subs_df = pd.read_csv('./csv_files/dndnext.csv')
dndnext_subs_df.head(3)

,subreddit,author,title,selftext,permalink,created_utc
0,dndnext,mellowOats,RAW the Drakewarden can totally use their drak...,So I was reading [this post](https://www.reddi...,/r/dndnext/comments/qh2037/raw_the_drakewarden...,1635356098
1,dndnext,Backflip248,"Does ""True Dragon"" have meaning anymore?",Question is in the title. With Fizban out and ...,/r/dndnext/comments/qh1tyr/does_true_dragon_ha...,1635355614
2,dndnext,coldhandsbigdick,How do people have fun RPing something other t...,I struggle with this *so much*. It's not that ...,/r/dndnext/comments/qh1tvh/how_do_people_have_...,1635355608


In [24]:
dmacademy_subs_df = pd.read_csv('./csv_files/dmacademy.csv')
dmacademy_subs_df.head(3)

,subreddit,author,title,selftext,permalink,created_utc
0,DMAcademy,TheJackofHats,Non-Combat Travel encounters,Ive been running several sessions of my player...,/r/DMAcademy/comments/qh1w02/noncombat_travel_...,1635355778
1,DMAcademy,El-Ahrairah7,Running an epic encounter where a town is dest...,"Hey all,\nI am about to attempt to run an enco...",/r/DMAcademy/comments/qh1ttr/running_an_epic_e...,1635355603
2,DMAcademy,Hellspore,Worried about my new adventure idea,I was wondering if we have a subreddit where o...,/r/DMAcademy/comments/qh1s9v/worried_about_my_...,1635355485


In [25]:
df = pd.concat([dndnext_subs_df, dmacademy_subs_df], ignore_index = True)
df.to_csv('./csv_files/submissions_orginal.csv', index=False)